## Necessary Library Imports

In [1]:
from piltonumpy_helper import pil_to_numpy
import os; import keras; import numpy as np; import pandas as pd; import shutil; from PIL import Image; import cv2
from tqdm import tqdm
from skimage.transform import resize, rescale

import matplotlib.pyplot as plt; import seaborn as sns;

from keras.layers import *
from keras.models import *

from tensorflow.keras.layers import Add
from keras.preprocessing.image import ImageDataGenerator

## Data Preprocessing

In [2]:
DATA_DIR = '/kaggle/input/the-car-connection-picture-dataset'
LOW_RES = '/kaggle/working/train_lowres'

In [3]:
os.mkdir('/kaggle/working/train_lowres')

In [4]:
os.chdir('/kaggle/working/train_lowres')

In [5]:
files = os.listdir(DATA_DIR)

In [6]:
len(files)

64467

In [7]:
files.sort()

In [8]:
import multiprocessing
from multiprocessing import Process

In [9]:

def make_low_res(files):
    for file in tqdm(files):
        img = Image.open(DATA_DIR + '/' + file)
        img = pil_to_numpy(img)
        #img = img/255.0
        img = cv2.resize(img, (256, 256))
        img = cv2.resize(cv2.resize(img, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_CUBIC), None, fx = 2, fy = 2, interpolation = cv2.INTER_CUBIC)
        im = Image.fromarray(img)
        im.save(file)
    

In [10]:
import time

In [11]:
# Making the low resolution folder in parallel processing

start = time.time()

p1 = Process(target = make_low_res, args = (files[:15000],))
p2 = Process(target = make_low_res, args = (files[15000:30000],))
p3 = Process(target = make_low_res, args = (files[30000:45000],))
p4 = Process(target = make_low_res, args = (files[45000:60000],))
p5 = Process(target = make_low_res, args = (files[60000:],))

p1.start()
p2.start()
p3.start()
p4.start()
p5.start()

p1.join()
p2.join()
p3.join()
p4.join()
p5.join()

end = time.time()

print('time elapsed -> ', end - start)

100%|██████████| 15000/15000 [04:57<00:00, 50.41it/s]


time elapsed ->  297.71193504333496


In [12]:
len(os.listdir('/kaggle/working/train_lowres'))

64467

### Make Datagenerators

In [13]:


def fetch_data_generator(files, batch_size = 64):
    while True:
        #
        batch_files = np.random.choice(files, batch_size)
        
        batch_x = [] ; batch_y = [];
        
        for file in batch_files:
            img = cv2.resize(pil_to_numpy(Image.open(DATA_DIR + '/' + file)).astype(float), (256,256))
            img_low = pil_to_numpy(Image.open(LOW_RES + '/' + file)).astype(float)
            
            batch_x.append(img_low/255.0)
            batch_y.append(img/255.0)
        
                
        yield np.array(batch_x), np.array(batch_y)

## Model Creation

Encoder Network

In [14]:
"""encoder = Sequential()
encoder.add(Conv2D(64, (3,3) , padding = 'same', activation = 'relu', input_shape = (256, 256, 3)))
encoder.add(Conv2D(64, (3,3), padding = 'same', activation = 'relu'))
encoder.add(MaxPooling2D((2,2), padding = 'same'))
encoder.add(Dropout(0.3))
encoder.add(Conv2D(128, (3,3), padding = 'same', activation = 'relu'))
encoder.add(Conv2D(128, (3,3), padding = 'same', activation = 'relu'))
encoder.add(MaxPooling2D((2,2), padding = 'same'))
encoder.add(Conv2D(256, (3,3), padding = 'same', activation = 'relu'))
encoder.summary()"""

"encoder = Sequential()\nencoder.add(Conv2D(64, (3,3) , padding = 'same', activation = 'relu', input_shape = (256, 256, 3)))\nencoder.add(Conv2D(64, (3,3), padding = 'same', activation = 'relu'))\nencoder.add(MaxPooling2D((2,2), padding = 'same'))\nencoder.add(Dropout(0.3))\nencoder.add(Conv2D(128, (3,3), padding = 'same', activation = 'relu'))\nencoder.add(Conv2D(128, (3,3), padding = 'same', activation = 'relu'))\nencoder.add(MaxPooling2D((2,2), padding = 'same'))\nencoder.add(Conv2D(256, (3,3), padding = 'same', activation = 'relu'))\nencoder.summary()"

Decoder Network

the decoder network would be the extension of the encoder 

In [15]:
#autoencoder = encoder

In [16]:
#autoencoder.summary()

In [17]:
# encoder

keras.backend.set_image_data_format('channels_last')

i1 = Input(shape = (256,256,3))
l1 = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(i1)
l2 = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(l1)
l3 = MaxPooling2D(padding = 'same')(l2)
l3 = Dropout(0.3)(l3)
l4 = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(l3)
l5 = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(l4)
l6 = MaxPooling2D(padding = 'same')(l5)
l7 = Conv2D(256, (3,3), padding = 'same', activation = 'relu')(l6)

# decoder

l8 = UpSampling2D()(l7)
l9 = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(l8)
l10 = Conv2D(128, (3,3), padding = 'same', activation = 'relu')(l9)
l11 = Add()([l5, l10])
l12 = UpSampling2D()(l11)
l13 = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(l12)
l14 = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(l13)
l15 = Add()([l14, l2])

# final layer should have 3 channels which will help to reconstruct the image with better resolution
l16 = Conv2D(3, (3,3), padding = 'same', activation = 'relu')(l15)

autoencoder = Model(i1, l16)

In [18]:
autoencoder.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [19]:
autoencoder.compile(optimizer = 'adadelta', loss = 'mean_squared_error')

## Training the Model

In [20]:
autoencoder.fit_generator(fetch_data_generator(files, 32), steps_per_epoch = len(files)//32, epochs = 10)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
2014/2014 [==============================] - 936s 462ms/step - loss: 0.2785
Epoch 2/10
2014/2014 [==============================] - 930s 462ms/step - loss: 0.0108
Epoch 3/10
2014/2014 [==============================] - 930s 462ms/step - loss: 0.0081
Epoch 4/10
2014/2014 [==============================] - 931s 462ms/step - loss: 0.0066
Epoch 5/10
2014/2014 [==============================] - 933s 463ms/step - loss: 0.0056
Epoch 6/10
2014/2014 [==============================] - 933s 463ms/step - loss: 0.0048
Epoch 7/10
2014/2014 [==============================] - 947s 470ms/step - loss: 0.0044
Epoch 8/10
2014/2014 [==============================] - 930s 462ms/step - loss: 0.0040
Epoch 9/10
2014/2014 [==============================] - 929s 461ms/step - loss: 0.0037
Epoch 10/10
2014/2014 [==============================] - 931s 462ms/step - loss: 0.0034


In [21]:
autoencoder.save('trained_10epochs.h5')

In [22]:
# Load Model
# uncomment to load the model with supplied weights & config, i.e., hdf5

#trained_model = keras.models.load_model('my_model.h5')